# RetrievalQA Chain

Developing a production-grade LLM application requires many refinements, but tracking multiple versions of prompts, models, and other components can be cumbersome. The [LangChain Hub](https://smith.langchain.com/hub) offers a centralized registry to manage and version your LLM artifacts efficiently. It even lets you interact with these artifacts directly in the browser to facilitate easier collaboration with non-technical team members.

<a href="https://smith.langchain.com/hub/rlm/rag-prompt/playground" target="_blank"><img src="./img/playground.png" alt="Playground" style="width:75%"></a>

In its initial release (08/05/2023), the hub is limited to prompt management, but we plan to add support for other artifacts soon.

In this walkthrough, you will get started using the hub to manage prompts for a retrieval QA chain. You will go through the following steps:

1. Load prompt from Hub
2. Initialize Chain
3. Run Chain
4. Commit any new changes to the hub

## Prerequsites

#### a. Set up your LangSmith account

While you can access public prompts without an account, pushing new prompts to the hub requires a LangSmith account. Create your account at https://smith.langchain.com and log in.

Next, navigate to the [hub home](https://smith.langchain.com/hub). If you haven't already created a "handle", you will be prompted to do so. Your prompts and other artifacts will be saved within the namespace '<handle>/prompt-name', so choose one that you are comfortable sharing.

#### b. Configure environment

To use the hub, you'll want to use a recent version of LangChain and the `langchainhub` package. Install them with the following command:

In [1]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


Finally, generate an API Key from your "personal" organization by navigating to the [LangSmith](https://smith.langchain.com) dashboard, and then set it in the cell below.

**Note:** Currently (08/04/2023), only API keys from your 'personal' organization are supported! If you see a '403' error at any point in this walkthrough, please confirm you've set a valid API key.

In [ ]:
%env LANGCHAIN_HUB_API_KEY=ls_...

## 1. Load prompt

Now it's time to load the prompt from the hub. We will use the `latest` version of [this retrieval QA prompt](https://smith.langchain.com/hub/rlm/rag-prompt) and later initialize the chain with it.

In [34]:
# RAG prompt
from langchain import hub

# Loads the latest version
prompt = hub.pull("rlm/rag-prompt")

# To load a specific version, specify the version hash
# prompt = hub.pull("rlm/rag-prompt:50442af1")

## 2. Create the QA chain

Now that we've selected our prompt, initialize the chain.
 For this example, we will create a basic [RetrievalQA](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval_qa.base.RetrievalQA.html?highlight=retrievalqa#langchain.chains.retrieval_qa.base.RetrievalQA) over a vectorstore retriever. 

Loading the data requires some amount of boilerplate, which we will run below.  While the specifics aren't important to this tutorial, you can learn more about Q&A in LangChain by visiting the [docs](https://python.langchain.com/docs/use_cases/question_answering/).

In [ ]:
# %pip install chroma-db --quiet

In [4]:
# Load docs
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

# Store splits
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# LLM
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

**Initialize the chain**. With the data added to the vectorstore, we can initialize the chain. We will
pass the prompt in via the `chain_type_kwargs` argument.

In [5]:
# RetrievalQA
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

'The approaches to task decomposition include using LLM with simple prompting, task-specific instructions, and human inputs.'

## 3. Run Chain

Now that the chain is initialized, you can run it just like you would normally.

In [ ]:
question = "What are the approaches to Task Decomposition?"
result = qa_chain({"query": question})
result["result"]

## 4. (Optional) Commit any new changes to the hub

After debugging, evaluating, or monitoring your chain in some deployment, you may want to make some changes to the prompt. You can do so by adding this prompt under your handle's namespace.

**Note:** If you receive a '403' forbidden error, you may need to set your `LANGCHAIN_HUB_API_KEY` to a personal API key.

In [35]:
# %env LANGCHAIN_HUB_API_KEY=ls_...

In [16]:
handle="wfh" # Replace with your handle!
hub.push(f"{handle}/rag-prompt", prompt)

'https://smith.langchain.com/hub/wfh/rag-prompt/c9839f14'

Now you can view your prompt in the hub. It should look something like this:

<a href="https://smith.langchain.com/hub/wfh/rag-prompt" target="_blank"><img src="./img/initial_push.png" alt="Initial push" style="width:75%"></a>

Let's say you've tried this prompt out and have derived a better one for your use case.
You can push the updated prompt to the same key to "commit" a new version of the prompt.

For instance, let's add a system message to the prompt:

In [28]:
# You may try making other changes and saving them in a new commit.
from langchain import schema

prompt.messages.insert(0, 
   schema.SystemMessage(
       content="You are a precise, autoregressive question-answering system."
   )
  )

In [32]:
# Pushing to the same prompt "repo" will create a new commit
hub.push(f"{handle}/rag-prompt", prompt)

Now the newest version of the prompt is saved as the `latest` version. It should look something like this:

<a href="https://smith.langchain.com/hub/wfh/rag-prompt" target="_blank"><img src="./img/updated.png" alt="Updated Prompt" style="width:75%"></a>

You can view all saved versions by navigating to the "commits" tab.

<a href="https://smith.langchain.com/hub/wfh/rag-prompt?tab=1" target="_blank"><img src="./img/commits.png" alt="Commits" style="width:75%"></a>

## Conclusion

In this tutorial, you learned how to use the [hub](https://smith.langchain.com/hub?page=1) to manage prompts for a retrieval QA chain. The hub is a centralized location to manage, version, and share your prompts (and later, other artifacts).

For more information, check out the [docs](https://docs.smith.langchain.com/category/hub) or reach out to support@langchain.dev.